## 3.1 EntropyMeasureV1

The instance `EntropyMeasure` contains 2 methods, one is `EntropyHadamardTest` using Hadamard test, and the other is `EntropyRandomizedMeasure` using randomized measure, the default case.

Both of them are used to calculate the entropy of a quantum state, but the calculation and post-processing are totally different.

`EntropyRandomizedMeasureV1` is a deprecated version of `EntropyRandomizedMeasure` because it lacks clarity on the relationship between bitstrings and qubit indices. 
This limitation can easily cause confusion, especially when working with systems that do not measure all qubits. However, it is retained for backward compatibility.

### 3.1.3 Randomized Measurement V1

This method is based on [Probing Rényi entanglement entropy via randomized measurements](https://www.science.org/doi/abs/10.1126/science.aau4963) with deplorizing error mitigation by [Simple mitigation of global depolarizing errors in quantum simulations](https://link.aps.org/doi/10.1103/PhysRevE.104.035309).


---

### b. `randomized_entangled_entropy_mitigated_v1`

<a id='randomized_entangled_entropy_mitigated_v1'></a>


Similar to `randomized_entangled_entropy` introduced in section 3.1.3 a, this function is used to calculate the entropy of a quantum state, but additionally, mitigate the depolarizing error. You can call the function `randomized_entangled_entropy_mitigated_v1` from `qurry.process.randomized_measure`. In order to perform the error mitigation, this function requires the  measurement outcomes of the all system as the baseline.  If you already have data at hand, you can prepare a `dict` like following:

```python
class ExistingAllSystemSource(TypedDict):
    """Existing all system source."""

    purityCellsAllSys: dict[int, float]
    """The purity of each cell of all system."""
    bitStringRange: tuple[int, int]
    """The range of partition on the bitstring."""
    measureActually: tuple[int, int]
    """The range of partition refer to all qubits."""
    source: str
    """The source of all system."""
```

And assign it into the argument `existed_all_system` of the function `randomized_entangled_entropy_mitigated_v1` to save a lot of time on mitigating purities of all partitions.

In this function, only `shots`, `counts`, and `degree` are required arguments. The others are optional.

The following is the arguments of the function:

```python
def randomized_entangled_entropy_mitigated_v1(
    shots: int,
    counts: list[dict[str, int]],
    degree: Optional[Union[tuple[int, int], int]],
    measure: Optional[tuple[int, int]] = None,
    backend: PostProcessingBackendLabel = DEFAULT_PROCESS_BACKEND,
    workers_num: Optional[int] = None,
    existed_all_system: Optional[ExistingAllSystemSource] = None,
    pbar: Optional[tqdm.tqdm] = None,
) -> RandomizedEntangledEntropyMitigatedComplex:
    """Calculate entangled entropy.

    Args:
        shots (int):
            Shots of the counts.
        counts (list[dict[str, int]]):
            Counts from randomized measurement results.
        degree (Optional[Union[tuple[int, int], int]]):
            The range of partition.
        measure (Optional[tuple[int, int]], optional):
            The range that implemented the measuring gate.
            If not specified, then use all qubits.
            This will affect the range of partition
            when you not implement the measuring gate on all qubit.
            Defaults to None.
        backend (PostProcessingBackendLabel, optional):
            Backend for the post-processing.
            Defaults to DEFAULT_PROCESS_BACKEND.
        workers_num (Optional[int], optional):
            Number of multi-processing workers, it will be ignored if backend is Rust.
            if sets to 1, then disable to using multi-processing;
            if not specified, then use the number of all cpu counts by `os.cpu_count()`.
            This only works for Python and Cython backend.
            Defaults to None.
        existed_all_system (Optional[ExistingAllSystemSource], optional):
            Existing all system source.
            If there is known all system result,
            then you can put it here to save a lot of time on calculating all system
            for not matter what partition you are using,
            their all system result is the same.
            All system source should contain
            `purityCellsAllSys`, `bitStringRange`, `measureActually`, `source` for its name.
            This can save a lot of time
            Defaults to None.
        pbar (Optional[tqdm.tqdm], optional):
            The progress bar API, you can use put a :cls:`tqdm` object here.
            This function will update the progress bar description.
            Defaults to None.

    Returns:
        RandomizedEntangledEntropyMitigatedComplex:
            A dictionary contains purity, entropy,
            a list of each overlap, puritySD, degree,
            actual measure range, bitstring range and more.
    """
```
This function returns a dictionary that contains the entropy, purity, and other relevant information.
The return `dict` contains the following variables:

```python
class RandomizedEntangledEntropyMitigatedComplex(TypedDict):
    """The result of the analysis."""

    purity: Union[np.float64, float]
    """The purity of the system."""
    entropy: Union[np.float64, float]
    """The entropy of the system."""
    puritySD: Union[np.float64, float]
    """The standard deviation of the purity."""
    entropySD: Union[np.float64, float]
    """The standard deviation of the entropy."""
    purityCells: Union[dict[int, np.float64], dict[int, float]]
    """The purity of each cell."""
    bitStringRange: Union[tuple[int, int], tuple[()]]
    """The range of partition on the bitstring."""

    allSystemSource: Union[str, Literal["independent"]]
    """The source of all system."""
    purityAllSys: Union[np.float64, float]
    """The purity of all system."""
    entropyAllSys: Union[np.float64, float]
    """The entropy of all system."""
    puritySDAllSys: Union[np.float64, float]
    """The standard deviation of the purity of all system."""
    entropySDAllSys: Union[np.float64, float]
    """The standard deviation of the entropy of all system."""
    purityCellsAllSys: Union[dict[int, np.float64], dict[int, float]]
    """The purity of each cell of all system."""
    bitsStringRangeAllSys: Union[tuple[int, int], tuple[()], None]
    """The range of partition on the bitstring of all system."""

    errorRate: Union[np.float64, float]
    """The error rate of the measurement from depolarizing error migigation calculated."""
    mitigatedPurity: Union[np.float64, float]
    """The mitigated purity."""
    mitigatedEntropy: Union[np.float64, float]
    """The mitigated entropy."""

    degree: Optional[Union[tuple[int, int], int]]
    """The range of partition."""
    num_qubits: int
    """The number of qubits of this syystem."""
    measure: tuple[str, Union[list[int], tuple[int, int]]]
    """The qubit range of the measurement and text description.

        - The first element is the text description.
        - The second element is the qubit range of the measurement.

        ---
        - When the measurement is specified, it will be:

        >>> ("measure range:", (0, 3))

        - When the measurement is not specified, it will be:

        >>> ("not specified, use all qubits", (0, 3))

        - When null counts exist, it will be:

        >>> ("The following is the index of null counts.", [0, 1, 2, 3])

        """
    measureActually: Union[tuple[int, int], tuple[()]]
    """The range of partition refer to all qubits."""
    measureActuallyAllSys: Union[tuple[int, int], tuple[()], None]
    """The range of partition refer to all qubits of all system."""

    countsNum: int
    """The number of counts."""
    takingTime: Union[np.float64, float]
    """The time of taking during specific partition."""
    takingTimeAllSys: Union[np.float64, float]
    """The taking time of the all system if it is calculated,
    it will be 0 when use the all system from other analysis.
    """
```



---

#### Dummy Data


In [1]:
from qurry.capsule import quickRead

easy_dummy: dict[str, dict[str, int]] = quickRead("./easy-dummy.json")
large_dummy_list = [easy_dummy["0"] for _ in range(100)]

---

#### Simple Example


In [2]:
from qurry.process.randomized_measure import (
    randomized_entangled_entropy_mitigated_v1,
    RandomizedEntangledEntropyMitigatedComplex,
    ExistingAllSystemSource,
)

test_result_1_2_1 = randomized_entangled_entropy_mitigated_v1(4096, large_dummy_list, 6)

In [3]:
from pprint import pprint

print("| result of randomized_entangled_entropy except for purityCells")
pprint({k: v for k, v in test_result_1_2_1.items() if "purityCells" not in k})
# "purityCells" is too long we skip it here
print()
print("| result of randomized_entangled_entropy[purityCells]")
print(test_result_1_2_1["purityCells"][0])
print(test_result_1_2_1["purityCells"][1])
print()
print("| result of randomized_entangled_entropy[purityCellsAllSys]")
print(test_result_1_2_1["purityCellsAllSys"][0])
print(test_result_1_2_1["purityCellsAllSys"][1])

| result of randomized_entangled_entropy except for purityCells
{'allSystemSource': 'independent',
 'bitStringRange': (2, 8),
 'bitsStringRangeAllSys': (0, 8),
 'countsNum': 100,
 'degree': 6,
 'entropy': -0.0878606530863832,
 'entropyAllSys': 0.9461940705953849,
 'entropySD': 0.0,
 'entropySDAllSys': 0.0,
 'errorRate': 0.2808939301105586,
 'measure': ('not specified, use all qubits', (2, 8)),
 'measureActually': (2, 8),
 'measureActuallyAllSys': (0, 8),
 'mitigatedEntropy': -1.0290289944568636,
 'mitigatedPurity': 2.0406503299038543,
 'num_qubits': 8,
 'purity': 1.0627930164337158,
 'purityAllSys': 0.5189998149871826,
 'puritySD': 0.0,
 'puritySDAllSys': 0.0,
 'takingTime': 0.003617926,
 'takingTimeAllSys': 0.010865806}

| result of randomized_entangled_entropy[purityCells]
1.0627930164337158
1.0627930164337158

| result of randomized_entangled_entropy[purityCellsAllSys]
0.5189998149871826
0.5189998149871826


---

#### With Existing All System Data


In [4]:
test_result_1_2_2 = randomized_entangled_entropy_mitigated_v1(
    4096,
    large_dummy_list,
    6,
    existed_all_system={
        "purityCellsAllSys": test_result_1_2_1["purityCellsAllSys"],
        "bitStringRange": test_result_1_2_1["bitStringRange"],
        "measureActually": test_result_1_2_1["measureActually"],
        "source": "from_previous_result:test_result_1_2_1",
    },
)

In [5]:
from pprint import pprint

print("| result of randomized_entangled_entropy except for purityCells")
pprint({k: v for k, v in test_result_1_2_2.items() if "purityCells" not in k})
# "purityCells" is too long we skip it here
print()
print("| result of randomized_entangled_entropy[purityCells]")
print(test_result_1_2_2["purityCells"][0])
print(test_result_1_2_2["purityCells"][1])
print()
print("| result of randomized_entangled_entropy[purityCellsAllSys]")
print(test_result_1_2_2["purityCellsAllSys"][0])
print(test_result_1_2_2["purityCellsAllSys"][1])
print()
print("| You can see takingTimeAllSys is 0 for we use existed_all_system")
print(test_result_1_2_2["takingTimeAllSys"])

| result of randomized_entangled_entropy except for purityCells
{'allSystemSource': 'from_previous_result:test_result_1_2_1',
 'bitStringRange': (2, 8),
 'bitsStringRangeAllSys': (2, 8),
 'countsNum': 100,
 'degree': 6,
 'entropy': -0.0878606530863832,
 'entropyAllSys': 0.9461940705953849,
 'entropySD': 0.0,
 'entropySDAllSys': 0.0,
 'errorRate': 0.2808939301105586,
 'measure': ('not specified, use all qubits', (2, 8)),
 'measureActually': (2, 8),
 'measureActuallyAllSys': (2, 8),
 'mitigatedEntropy': -1.0290289944568636,
 'mitigatedPurity': 2.0406503299038543,
 'num_qubits': 8,
 'purity': 1.0627930164337158,
 'purityAllSys': 0.5189998149871826,
 'puritySD': 0.0,
 'puritySDAllSys': 0.0,
 'takingTime': 0.002607396,
 'takingTimeAllSys': 0}

| result of randomized_entangled_entropy[purityCells]
1.0627930164337158
1.0627930164337158

| result of randomized_entangled_entropy[purityCellsAllSys]
0.5189998149871826
0.5189998149871826

| You can see takingTimeAllSys is 0 for we use existed_all_

---

#### Integration wit your own progress bar


In [6]:
from tqdm import tqdm


all_counts_progress_01 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)

test_result_1_2_3 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_01:
    test_result_1_2_3.append(
        randomized_entangled_entropy_mitigated_v1(
            tmp_shot,
            tmp_counts,
            tmp_partition,
            existed_all_system=(
                None
                if len(test_result_1_2_3) == 0
                else {
                    "purityCellsAllSys": test_result_1_2_3[-1]["purityCellsAllSys"],
                    "bitStringRange": test_result_1_2_3[-1]["bitStringRange"],
                    "measureActually": test_result_1_2_3[-1]["measureActually"],
                    "source": "from_previous_result:test_result_1_2_1",
                }
            ),
            pbar=all_counts_progress_01,
        )
    )
    print(f"| partition: {tmp_partition}")
    print("| - takingTime:", test_result_1_2_3[-1]["takingTime"])
    print("| - takingTimeAllSys:", test_result_1_2_3[-1]["takingTimeAllSys"])

| Calculate specific degree (-2, 5) by Rust. - 00:00 < 00:00                             

| partition: 6
| - takingTime: 0.004006246
| - takingTimeAllSys: 0.014917189
| partition: (2, 8)
| - takingTime: 0.02929423
| - takingTimeAllSys: 0
| partition: 7
| - takingTime: 0.037732631
| - takingTimeAllSys: 0
| partition: (0, 7)
| - takingTime: 0.030506098
| - takingTimeAllSys: 0


| Calculate specific degree (-5, -1) by Rust. - 00:00 < 00:00                            

| partition: (-2, 5)
| - takingTime: 0.029499891
| - takingTimeAllSys: 0


| Preparing error mitigation of (3, 6) on None - 00:00 < 00:00                           

| partition: (-5, -1)
| - takingTime: 0.008938533
| - takingTimeAllSys: 0


| urce is not in existed_all_system with mitigation. - 00:00 < 00:00

| partition: (3, -2)
| - takingTime: 0.011057409
| - takingTimeAllSys: 0


---

#### Using Python backend

It will be slow. Yoy better think twice before using it.


In [7]:
all_counts_progress_02 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)


test_result_1_2_4 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_02:
    test_result_1_2_4.append(
        randomized_entangled_entropy_mitigated_v1(
            tmp_shot,
            tmp_counts,
            tmp_partition,
            existed_all_system=(
                None
                if len(test_result_1_2_4) == 0
                else {
                    "purityCellsAllSys": test_result_1_2_4[-1]["purityCellsAllSys"],
                    "bitStringRange": test_result_1_2_4[-1]["bitStringRange"],
                    "measureActually": test_result_1_2_4[-1]["measureActually"],
                    "source": "from_previous_result:test_result_1_2_4",
                }
            ),
            pbar=all_counts_progress_02,
            backend="Python",
        )
    )

| urce is not in existed_all_system with mitigation. - 00:07 < 00:00                                            


---

### Post-Process Availablities and Version Info



In [8]:
from qurry.process.status import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.9.2.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap ................... Yes    Depr.  Yes   
   - echo_cell .............................. Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally.
   + Error .... Exception occurred.
   + No ....... Not suppo